In [ ]:
import random

# --- Simulation Parameters ---
# You can change these values to simulate different characters.
HIT_DIE = 8          # e.g., 8 for a d8 (Cleric/Rogue), 10 for a d10 (Fighter/Paladin)
CON_MODIFIER = 2     # The character's Constitution modifier.
MAX_LEVEL = 20       # The maximum level to simulate.
NUM_SIMULATIONS = 10000 # Number of times to run the simulation for stats.

# --- Calculated Constants ---
# The "average" HP gain per level, rounded up. For a d8, (8/2)+0.5 = 4.5 -> 5
AVERAGE_HP_GAIN = int(HIT_DIE / 2) + 1

def get_average_hp_at_level(level):
    """Calculates the expected HP at a given level using the 'average' method."""
    if level <= 0:
        return 0
    if level == 1:
        return HIT_DIE + CON_MODIFIER
    # Max HP at level 1 + (level-1) times the average gain
    return (HIT_DIE + CON_MODIFIER) + ((level - 1) * (AVERAGE_HP_GAIN + CON_MODIFIER))

def run_full_simulation():
    """
    Runs a multi-level simulation to gather statistics on HP outcomes
    at each level for the different methods.
    """
    print("--- Running Simulations ---")
    print(f"Character: d{HIT_DIE} Hit Die, +{CON_MODIFIER} CON Modifier")
    print(f"Running {NUM_SIMULATIONS} simulations for each level up to {MAX_LEVEL}.\n")

    # This will store our results for each level
    results = []

    for level in range(1, MAX_LEVEL + 1):
        # Counters for statistical outcomes at the current level
        rolling_significantly_below = 0
        rolling_significantly_above = 0
        rolling_two_levels_ahead = 0
        rerolling_significantly_below = 0
        rerolling_significantly_above = 0
        rerolling_two_levels_ahead = 0

        # Get the baseline average HP for the current level and thresholds
        avg_hp_baseline = get_average_hp_at_level(level)
        avg_hp_two_levels_ahead = get_average_hp_at_level(level + 2)
        hp_threshold = AVERAGE_HP_GAIN

        if level == 1:
            # At level 1, all methods yield the same result.
            results.append({
                "level": 1, "avg_hp": avg_hp_baseline,
                "roll_below_%": 0, "roll_above_%": 0, "roll_2_ahead_%": 0,
                "reroll_below_%": 0, "reroll_above_%": 0, "reroll_2_ahead_%": 0
            })
            continue

        # Run the simulation NUM_SIMULATIONS times for the current level
        for _ in range(NUM_SIMULATIONS):
            # --- Simulate Standard Rolling ---
            hp_rolling = HIT_DIE + CON_MODIFIER
            for lvl_up in range(2, level + 1):
                hp_rolling += random.randint(1, HIT_DIE) + CON_MODIFIER

            if hp_rolling < avg_hp_baseline - hp_threshold:
                rolling_significantly_below += 1
            elif hp_rolling > avg_hp_baseline + hp_threshold:
                rolling_significantly_above += 1
            
            # This check now works for all levels
            if hp_rolling >= avg_hp_two_levels_ahead:
                rolling_two_levels_ahead += 1

            # --- Simulate Rerolling All Dice ---
            hit_dice_rolls = [HIT_DIE]
            for reroll_lvl in range(2, level + 1):
                new_rolls = [random.randint(1, HIT_DIE) for _ in range(reroll_lvl)]
                avg_hp_baseline_at_reroll_lvl = get_average_hp_at_level(reroll_lvl)
                new_hp_total = sum(new_rolls) + (reroll_lvl * CON_MODIFIER)

                if new_hp_total > (avg_hp_baseline_at_reroll_lvl + hp_threshold):
                    hit_dice_rolls = new_rolls
                else:
                    hit_dice_rolls.append(random.randint(1, HIT_DIE))
            
            final_reroll_hp = sum(hit_dice_rolls) + (level * CON_MODIFIER)

            if final_reroll_hp < avg_hp_baseline - hp_threshold:
                rerolling_significantly_below += 1
            elif final_reroll_hp > avg_hp_baseline + hp_threshold:
                rerolling_significantly_above += 1
            
            # This check now works for all levels
            if final_reroll_hp >= avg_hp_two_levels_ahead:
                rerolling_two_levels_ahead += 1
        
        # Calculate percentages
        results.append({
            "level": level, "avg_hp": avg_hp_baseline,
            "roll_below_%": (rolling_significantly_below / NUM_SIMULATIONS) * 100,
            "roll_above_%": (rolling_significantly_above / NUM_SIMULATIONS) * 100,
            "roll_2_ahead_%": (rolling_two_levels_ahead / NUM_SIMULATIONS) * 100,
            "reroll_below_%": (rerolling_significantly_below / NUM_SIMULATIONS) * 100,
            "reroll_above_%": (rerolling_significantly_above / NUM_SIMULATIONS) * 100,
            "reroll_2_ahead_%": (rerolling_two_levels_ahead / NUM_SIMULATIONS) * 100
        })

    # --- Print the final results table ---
    print("--- Statistical Results ---")
    print("'% Below/Above' = Chance to be >1 avg die roll away from the mean.")
    print("'% 2 Ahead' = Chance to have HP >= the average of a character 2 levels higher.")
    print("-" * 110)
    print(f"{'Lvl':<4} | {'Avg HP':<7} | {'Roll %<':<9} | {'Roll %>':<9} | {'Roll %2Ah':<10} | {'Reroll %<':<10} | {'Reroll %>':<10} | {'Reroll %2Ah':<10}")
    print("-" * 110)

    for res in results:
        print(f"{res['level']:<4} | {res['avg_hp']:<7} | "
              f"{res['roll_below_%']:<8.2f}% | "
              f"{res['roll_above_%']:<8.2f}% | "
              f"{res['roll_2_ahead_%']:<9.2f}% | "
              f"{res['reroll_below_%']:<9.2f}% | "
              f"{res['reroll_above_%']:<9.2f}% | "
              f"{res['reroll_2_ahead_%']:<9.2f}%")
    print("-" * 110)


# --- Run the main function ---
if __name__ == "__main__":
    run_full_simulation()
